In [1]:
import polars as pl
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import numpy as np
import logging
import random

seed = 42
np.random.seed(seed)
random.seed(seed)
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.random.set_seed(seed)
print(tf.__version__)

2024-06-17 15:27:17.581768: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 15:27:17.656498: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 15:27:18.846365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.16.1


In [13]:
import numpy as np
from datetime import datetime, timedelta

def optimized_target_id(x_i, x, impression_time_idx, i, window):
    # Precompute timedelta once
    seven_days = timedelta(days=7)

    # Get the last impression time once
    last_window_date = x_i[impression_time_idx][-1]
    print(last_window_date)

    # Create a mask for valid time indices based on the 7 days condition
    time_diffs = x[:, (i + window + 1):][impression_time_idx] - last_window_date
    print(time_diffs)
    valid_mask = time_diffs <= seven_days

    # Find the maximum valid index within the mask
    valid_indices = np.where(valid_mask)[0] + (i + window + 1)
    print(valid_indices)

    # Determine max_telescope using the precomputed valid_indices
    max_telescope = valid_indices[-1] if valid_indices.size > 0 else i + window
    print(max_telescope)

    # Randomly select a target_id within the valid range
    target_id = np.random.randint(i + window, max_telescope + 1)

    return target_id


def non_optimized_target_id(x_i, x, impression_time_idx, i, window):
    # Precompute timedelta once
    seven_days = timedelta(days=7)

    # Get the last impression time once
    last_window_date = x_i[impression_time_idx][-1]

    # Precompute the valid time range indices
    valid_indices = [t for t in range(i + window + 1, x.shape[1])
                    if (x[:, t][impression_time_idx] - last_window_date) <= seven_days]

    # Determine max_telescope using the precomputed valid_indices
    max_telescope = max(valid_indices, default=i + window)

    # Randomly select a target_id within the valid range
    target_id = np.random.randint(i + window, max_telescope + 1)
    
    return target_id

# Test case
# Example data setup
impression_time_idx = 0
i = 0
window = 2

# Create dummy datetime data
base_time = datetime(2024, 1, 1)
x_i = np.array([
    [base_time, base_time + timedelta(days=1), base_time + timedelta(days=2),
        base_time + timedelta(days=3), base_time + timedelta(days=4), base_time + timedelta(days=5)]
])

# Replicate x_i data for x array, adding more columns for the time range
x = np.tile(x_i, (3, 1))  # Assuming 3 rows for demonstration

# Call the function
target_id = optimized_target_id(x_i, x, impression_time_idx, i, window)
target_id_a = non_optimized_target_id(x_i, x, impression_time_idx, i, window)
print("Selected target_id:", target_id, target_id_a)



2024-01-06 00:00:00
[datetime.timedelta(days=-2) datetime.timedelta(days=-1)
 datetime.timedelta(0)]
[3 4 5]
5
Selected target_id: 5 2


In [19]:
import numpy as np

arr = np.array([1, 2, 3, 4, 5])
condition = arr > 3
result = np.where(condition)
result[0]

array([3, 4])

In [20]:
result[0].size

2

In [23]:
history = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/history.parquet')
history_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/history.parquet')
history_testset = 
behaviors = pl.read_parquet('/home/ubuntu/dataset/ebnerd_large/train/behaviors.parquet')
behaviors_testset = pl.read_parquet('/home/ubuntu/dataset/ebnerd_testset/test/behaviors.parquet')
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/articles.parquet')
history.head(2)

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"


In [28]:
behaviors_testset.group_by('user_id').agg(pl.all().first()).describe()

statistic,user_id,impression_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,is_sso_user,gender,postcode,age,is_subscriber,session_id,is_beyond_accuracy
str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",807677.0,807677.0,"""807677""",807677.0,240229.0,807677.0,807677.0,807677.0,33435.0,10939.0,14777.0,807677.0,807677.0,807677.0
"""null_count""",0.0,0.0,"""0""",0.0,567448.0,0.0,0.0,0.0,774242.0,796738.0,792900.0,0.0,0.0,0.0
"""mean""",1.3006e6,1.2665e8,"""2023-06-04 14:03:03.409243""",51.282566,99.396301,1.651406,null,0.062682,0.150501,1.493372,50.280842,0.031769,6.7135e7,0.0
"""std""",745166.979895,1.3670e8,null,106.134048,5.766483,0.530605,null,null,0.357651,1.45708,14.701355,null,3.1864e7,null
"""min""",10000.0,6.451339e6,"""2023-06-01 07:00:00""",0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,44.0,0.0
"""25%""",655833.0,2.9926166e7,"""2023-06-02 14:48:36""",12.0,100.0,1.0,null,null,0.0,0.0,40.0,null,4.1347833e7,null
"""50%""",1.30141e6,7.4352964e7,"""2023-06-04 14:32:35""",23.0,100.0,2.0,null,null,0.0,1.0,50.0,null,7.4099885e7,null
"""75%""",1.945135e6,1.8755451e8,"""2023-06-06 09:58:34""",52.0,100.0,2.0,null,null,0.0,3.0,60.0,null,9.570464e7,null
"""max""",2.590693e6,5.75650165e8,"""2023-06-08 06:59:59""",1800.0,100.0,3.0,null,1.0,2.0,4.0,100.0,1.0,1.08582914e8,0.0


In [29]:
behaviors_testset.group_by('user_id').agg(pl.all().first())['age'].is_null().sum() / behaviors_testset.group_by('user_id').agg(pl.all().first())['age'].is_not_null().sum()

53.65771130811396

In [25]:
behaviors_testset.filter(pl.col('age').is_not_null())['age'].describe()

statistic,value
str,f64
"""count""",397679.0
"""null_count""",0.0
"""mean""",52.462564
"""std""",14.04399
"""min""",0.0
"""25%""",40.0
"""50%""",50.0
"""75%""",60.0
"""max""",100.0


In [19]:
from tqdm import tqdm
history_complete = pl.concat([
    slice.explode(pl.all().exclude('user_id'))\
        .sort('impression_time_fixed').unique('impression_time_fixed')\
        .group_by('user_id').agg(pl.all())
    for slice in tqdm(history.vstack(history_val).sort('user_id').partition_by('user_id', maintain_order=False))
])
history_complete

100%|██████████| 18827/18827 [00:28<00:00, 652.98it/s]


user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
10068,"[2023-04-27 14:22:26, 2023-04-27 18:51:53, … 2023-05-16 19:48:21]","[100.0, 100.0, … 100.0]","[9735753, 9739065, … 9759717]","[39.0, 21.0, … 7.0]"
10200,"[2023-05-14 05:46:27, 2023-05-14 05:46:44, … 2023-05-23 13:16:25]","[36.0, 24.0, … 25.0]","[9764325, 9763923, … 9777308]","[16.0, 15.0, … 6.0]"
10201,"[2023-04-27 09:49:02, 2023-04-27 09:49:47, … 2023-05-25 06:50:32]","[97.0, 55.0, … 100.0]","[9738557, 9738569, … 9779205]","[45.0, 16.0, … 33.0]"
10385,"[2023-05-23 11:46:06, 2023-05-23 11:46:26, … 2023-05-25 04:56:20]","[26.0, 29.0, … 32.0]","[9777320, 9777475, … 9780020]","[19.0, 25.0, … 9.0]"
10623,"[2023-04-27 14:29:44, 2023-04-27 14:29:47, … 2023-05-24 17:10:11]","[48.0, 100.0, … 32.0]","[9739035, 9739065, … 9779517]","[1.0, 9.0, … 2.0]"
…,…,…,…,…
2590039,"[2023-05-23 17:54:22, 2023-05-23 18:06:03, … 2023-05-24 19:12:10]","[null, 100.0, … 100.0]","[9778168, 9778198, … 9779737]","[701.0, 47.0, … 1.0]"
2590054,"[2023-04-30 18:04:22, 2023-04-30 18:05:03, … 2023-05-17 19:07:00]","[100.0, 99.0, … 44.0]","[9741288, 9743490, … 9764444]","[41.0, 10.0, … 14.0]"
2590471,"[2023-05-17 17:27:42, 2023-05-17 17:28:10, … 2023-05-18 01:13:42]","[87.0, 100.0, … 26.0]","[9770533, 9770491, … 9768321]","[23.0, 47.0, … 9.0]"


In [5]:
history['scroll_percentage_fixed'].list.len().describe()

statistic,value
str,f64
"""count""",15342.0
"""null_count""",0.0
"""mean""",143.669209
"""std""",165.740723
"""min""",5.0
"""25%""",30.0
"""50%""",81.0
"""75%""",193.0
"""max""",1126.0


In [7]:
history

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
14241,"[2023-05-04 12:42:38, 2023-05-04 12:43:04, … 2023-05-24 13:58:04]","[100.0, 77.0, … 100.0]","[9578072, 9749349, … 9779289]","[20.0, 16.0, … 1.0]"
20396,"[2023-05-04 10:09:26, 2023-05-04 10:47:17, … 2023-05-24 20:51:06]","[null, 100.0, … 11.0]","[9748482, 9748916, … 9779489]","[0.0, 69.0, … 8.0]"
37953,"[2023-05-04 20:51:41, 2023-05-04 20:51:55, … 2023-05-24 21:10:46]","[25.0, 13.0, … 97.0]","[9750397, 9749916, … 9778902]","[5.0, 5.0, … 13.0]"
38910,"[2023-05-04 13:41:54, 2023-05-04 13:41:57, … 2023-05-25 05:43:36]","[null, 23.0, … 100.0]","[9745491, 9749729, … 9779577]","[2.0, 2.0, … 71.0]"
39221,"[2023-05-06 03:54:52, 2023-05-06 03:56:00, … 2023-05-23 23:14:07]","[100.0, 100.0, … 24.0]","[9749034, 9751385, … 9778168]","[67.0, 82.0, … 3.0]"
…,…,…,…,…
906586,"[2023-05-07 07:03:15, 2023-05-13 19:19:21, … 2023-05-13 19:22:24]","[60.0, 100.0, … 100.0]","[9725978, 9763942, … 9763634]","[130.0, 37.0, … 83.0]"
2028720,"[2023-05-24 09:46:38, 2023-05-24 09:47:15, … 2023-05-24 09:49:43]","[100.0, 100.0, … 100.0]","[9778731, 9578072, … 9778219]","[37.0, 14.0, … 84.0]"
1455620,"[2023-05-24 15:31:37, 2023-05-24 15:33:10, … 2023-05-24 15:43:10]","[100.0, 100.0, … 100.0]","[9778819, 9778819, … 9779538]","[52.0, 55.0, … 14.0]"


In [6]:
behaviors

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
580099643,9769306,2023-05-18 10:01:05,121.0,100.0,3,"[9233208, 9771242, … 9521144]",[9770886],2106715,false,null,null,null,false,1416293,121.0,null
580099644,9770882,2023-05-18 10:05:07,176.0,100.0,3,"[9771065, 9767697, … 9769762]",[9769306],2106715,false,null,null,null,false,1416293,148.0,100.0
580099645,9769306,2023-05-18 10:11:03,24.0,100.0,3,"[9771042, 9440508, … 9767697]",[9771042],2106715,false,null,null,null,false,1416293,4.0,null


In [9]:
a = history['impression_time_fixed'].explode()[0]
b = history['impression_time_fixed'].explode()[100]
a, b, b-a

(datetime.datetime(2023, 4, 27, 10, 17, 43),
 datetime.datetime(2023, 5, 1, 3, 24, 5),
 datetime.timedelta(days=3, seconds=61582))

In [5]:
from datetime import timedelta
(b - a) <= timedelta(days=7, minutes=0, seconds=0)

True

In [43]:
history.shape[0]//4

3785

In [47]:
sampled_users = history.select('user_id', pl.col('article_id_fixed').list.len().alias('h_len'))
sampled_users = sampled_users.filter(
    pl.col('h_len') >= pl.col('h_len').quantile(0.25),
    pl.col('h_len') <= pl.col('h_len').quantile(0.75),   
)['user_id'].to_list()


(shape: (9, 3)
 ┌────────────┬───────────────┬────────────┐
 │ statistic  ┆ user_id       ┆ h_len      │
 │ ---        ┆ ---           ┆ ---        │
 │ str        ┆ f64           ┆ f64        │
 ╞════════════╪═══════════════╪════════════╡
 │ count      ┆ 7596.0        ┆ 7596.0     │
 │ null_count ┆ 0.0           ┆ 0.0        │
 │ mean       ┆ 1.3053e6      ┆ 104.080305 │
 │ std        ┆ 742325.744084 ┆ 52.026735  │
 │ min        ┆ 10201.0       ┆ 35.0       │
 │ 25%        ┆ 666694.0      ┆ 59.0       │
 │ 50%        ┆ 1.315152e6    ┆ 93.0       │
 │ 75%        ┆ 1.950222e6    ┆ 143.0      │
 │ max        ┆ 2.590637e6    ┆ 222.0      │
 └────────────┴───────────────┴────────────┘,
 59.0)

In [3]:
topics = articles['topics'].explode().unique().drop_nans().drop_nulls().sort().to_frame().with_row_index()
category = articles['category'].unique().drop_nans().drop_nulls().sort().to_frame().with_row_index(offset=1)
subcategory = articles['subcategory'].explode().unique().drop_nans().drop_nulls().sort().to_frame().with_row_index()
sentiment_label = articles['sentiment_label'].explode().unique().drop_nans().drop_nulls().sort().to_frame().with_row_index(offset=1)
mask = 0

articles = articles.select(['article_id', 'category', 'subcategory', 'premium', 'topics', 'sentiment_label'])\
    .with_columns(
        pl.col('topics').fill_null(pl.lit([])),
        pl.col('subcategory').fill_null(pl.lit([]))
    )\
    .with_columns(
        pl.col('topics').list.eval(pl.element().replace(topics['topics'], topics['index'], default=None)).list.drop_nulls().cast(pl.List(pl.Int32)),
        pl.col('category').replace(category['category'], category['index'], default=None).fill_null(mask).cast(pl.Int32),
        pl.col('sentiment_label').replace(sentiment_label['sentiment_label'], sentiment_label['index'], default=None).fill_null(mask).cast(pl.Int32),
        pl.col('subcategory').list.eval(pl.element().replace(subcategory['subcategory'], subcategory['index'], default=None)).list.drop_nulls().cast(pl.List(pl.Int32)),
        pl.col('premium').cast(pl.Int8)
)
articles.head(2)

article_id,category,subcategory,premium,topics,sentiment_label
i32,i32,list[i32],i8,list[i32],i32
3001353,5,[],0,"[25, 47]",1
3003065,7,"[87, 88]",0,"[69, 13, 77]",3


In [4]:
dummies_topics = articles.select('article_id', 'topics').explode('topics').drop_nulls().to_dummies(columns=['topics'])\
    .group_by('article_id').agg(pl.all().sum())
dummies_subcategories = articles.select('article_id', 'subcategory').explode('subcategory').drop_nulls().to_dummies(columns=['subcategory'])\
    .group_by('article_id').agg(pl.all().sum())
    
dummies_subcategories.head(2)

article_id,subcategory_0,subcategory_1,subcategory_10,subcategory_100,subcategory_101,subcategory_102,subcategory_103,subcategory_104,subcategory_105,subcategory_106,subcategory_107,subcategory_108,subcategory_109,subcategory_11,subcategory_110,subcategory_111,subcategory_112,subcategory_113,subcategory_114,subcategory_115,subcategory_116,subcategory_117,subcategory_118,subcategory_119,subcategory_12,subcategory_120,subcategory_121,subcategory_122,subcategory_123,subcategory_124,subcategory_125,subcategory_126,subcategory_127,subcategory_128,subcategory_129,subcategory_13,…,subcategory_66,subcategory_67,subcategory_68,subcategory_69,subcategory_7,subcategory_70,subcategory_71,subcategory_72,subcategory_73,subcategory_74,subcategory_75,subcategory_76,subcategory_77,subcategory_78,subcategory_79,subcategory_8,subcategory_80,subcategory_81,subcategory_82,subcategory_83,subcategory_84,subcategory_85,subcategory_86,subcategory_87,subcategory_88,subcategory_89,subcategory_9,subcategory_90,subcategory_91,subcategory_92,subcategory_93,subcategory_94,subcategory_95,subcategory_96,subcategory_97,subcategory_98,subcategory_99
i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
8901774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9736227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
from polimi.utils._polars import reduce_polars_df_memory_size

articles = articles.join(dummies_topics, on='article_id', how='left')\
    .join(dummies_subcategories, on='article_id', how='left')\
    .drop('topics', 'subcategory')
articles = reduce_polars_df_memory_size(articles)

one_hot_cols = [col for col in articles.columns if col.startswith('topics_') or col.startswith('subcategory_')]
articles = articles.with_columns(
    pl.col(one_hot_cols).fill_null(0)
)

articles.head(2)

Memory usage of dataframe is 40.75 MB
Memory usage after optimization is: 5.75 MB
Decreased by 85.9%


article_id,category,premium,sentiment_label,topics_0,topics_1,topics_10,topics_11,topics_12,topics_13,topics_14,topics_15,topics_16,topics_17,topics_18,topics_19,topics_2,topics_20,topics_21,topics_22,topics_23,topics_24,topics_25,topics_26,topics_27,topics_28,topics_29,topics_3,topics_30,topics_31,topics_32,topics_33,topics_34,topics_35,topics_36,topics_37,topics_38,…,subcategory_66,subcategory_67,subcategory_68,subcategory_69,subcategory_7,subcategory_70,subcategory_71,subcategory_72,subcategory_73,subcategory_74,subcategory_75,subcategory_76,subcategory_77,subcategory_78,subcategory_79,subcategory_8,subcategory_80,subcategory_81,subcategory_82,subcategory_83,subcategory_84,subcategory_85,subcategory_86,subcategory_87,subcategory_88,subcategory_89,subcategory_9,subcategory_90,subcategory_91,subcategory_92,subcategory_93,subcategory_94,subcategory_95,subcategory_96,subcategory_97,subcategory_98,subcategory_99
i32,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
3001353,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3003065,7,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df = pl.concat([
    slice.explode(pl.all().exclude('user_id'))\
        .with_columns(
            pl.col('scroll_percentage_fixed').fill_null(0.),
            pl.col('read_time_fixed').fill_null(0.),
        )\
        .with_columns(
            (pl.col('impression_time_fixed').dt.hour() // 4).alias('hour_group'),
            pl.col('impression_time_fixed').dt.weekday().alias('weekday'),
        )\
        .rename({'scroll_percentage_fixed': 'scroll_percentage', 'read_time_fixed': 'read_time'})
        .join(articles, left_on='article_id_fixed', right_on='article_id', how='left').drop('article_id_fixed')\
        .group_by('user_id').agg(pl.all())
    for slice in history.iter_slices(10000)
])

df.head(3)

user_id,impression_time_fixed,scroll_percentage,read_time,hour_group,weekday,category,premium,sentiment_label,topics_0,topics_1,topics_10,topics_11,topics_12,topics_13,topics_14,topics_15,topics_16,topics_17,topics_18,topics_19,topics_2,topics_20,topics_21,topics_22,topics_23,topics_24,topics_25,topics_26,topics_27,topics_28,topics_29,topics_3,topics_30,topics_31,topics_32,topics_33,…,subcategory_66,subcategory_67,subcategory_68,subcategory_69,subcategory_7,subcategory_70,subcategory_71,subcategory_72,subcategory_73,subcategory_74,subcategory_75,subcategory_76,subcategory_77,subcategory_78,subcategory_79,subcategory_8,subcategory_80,subcategory_81,subcategory_82,subcategory_83,subcategory_84,subcategory_85,subcategory_86,subcategory_87,subcategory_88,subcategory_89,subcategory_9,subcategory_90,subcategory_91,subcategory_92,subcategory_93,subcategory_94,subcategory_95,subcategory_96,subcategory_97,subcategory_98,subcategory_99
u32,list[datetime[μs]],list[f32],list[f32],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],…,list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8]
242896,"[2023-04-27 10:16:13, 2023-04-27 14:28:42, … 2023-05-17 19:53:04]","[34.0, 0.0, … 100.0]","[3.0, 25.0, … 6.0]","[2, 3, … 4]","[4, 4, … 3]","[6, 7, … 5]","[0, 0, … 0]","[1, 1, … 1]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"
1040907,"[2023-04-27 14:56:23, 2023-04-27 14:56:50, … 2023-05-18 04:14:20]","[67.0, 67.0, … 27.0]","[25.0, 5.0, … 4.0]","[3, 3, … 1]","[4, 4, … 4]","[4, 7, … 7]","[0, 0, … 0]","[2, 1, … 2]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"
1314996,"[2023-04-28 10:47:03, 2023-04-28 10:48:27, … 2023-05-17 19:14:39]","[100.0, 42.0, … 16.0]","[84.0, 5.0, … 11.0]","[2, 2, … 4]","[5, 5, … 3]","[5, 7, … 6]","[0, 0, … 0]","[1,

In [7]:
from polimi.utils._polars import reduce_polars_df_memory_size
cols = df.columns
topics_cols = sorted([col for col in cols if col.startswith('topics_')], key=lambda x: int(x.split('_')[-1]))
subcategory_cols = sorted([col for col in cols if col.startswith('subcategory_')], key=lambda x: int(x.split('_')[-1]))
all_others = set(cols) - set(topics_cols) - set(subcategory_cols) - {'user_id'}
cols = ['user_id'] + list(all_others) + topics_cols + subcategory_cols
df = df.select(cols)
df.head(1)

user_id,read_time,category,sentiment_label,impression_time_fixed,premium,hour_group,scroll_percentage,weekday,topics_0,topics_1,topics_2,topics_3,topics_4,topics_5,topics_6,topics_7,topics_8,topics_9,topics_10,topics_11,topics_12,topics_13,topics_14,topics_15,topics_16,topics_17,topics_18,topics_19,topics_20,topics_21,topics_22,topics_23,topics_24,topics_25,topics_26,topics_27,…,subcategory_137,subcategory_138,subcategory_139,subcategory_140,subcategory_141,subcategory_142,subcategory_143,subcategory_144,subcategory_145,subcategory_146,subcategory_147,subcategory_148,subcategory_149,subcategory_150,subcategory_151,subcategory_152,subcategory_153,subcategory_154,subcategory_155,subcategory_156,subcategory_157,subcategory_158,subcategory_159,subcategory_160,subcategory_161,subcategory_162,subcategory_163,subcategory_164,subcategory_165,subcategory_166,subcategory_167,subcategory_168,subcategory_169,subcategory_170,subcategory_171,subcategory_172,subcategory_173
u32,list[f32],list[i8],list[i8],list[datetime[μs]],list[i8],list[i8],list[f32],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],…,list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8]
242896,"[3.0, 25.0, … 6.0]","[6, 7, … 5]","[1, 1, … 1]","[2023-04-27 10:16:13, 2023-04-27 14:28:42, … 2023-05-17 19:53:04]","[0, 0, … 0]","[2, 3, … 4]","[34.0, 0.0, … 100.0]","[4, 4, … 3]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"


In [8]:
impression_time_idx = df.drop('user_id').columns.index('impression_time_fixed')
for user_df in df[:1].partition_by('user_id'):
        x = user_df.drop('user_id').to_numpy()[0]
        x = np.array([np.array(x_i) for x_i in x])
d = x[impression_time_idx][1]
d

datetime.datetime(2023, 4, 27, 14, 28, 42)

In [9]:
d.month

4

In [10]:
curr_date = x[:, 10][impression_time_idx]
last_step = [i for i in range(10, 20) if x[:, i][impression_time_idx].month == curr_date.month and x[:, i][impression_time_idx].day == curr_date.day]
last_step

[10, 11, 12]

In [16]:
from tqdm import tqdm

def build_sequences(df: pl.DataFrame, w: int, stride: int):
    all_features = df.drop('user_id').columns
    singular_cols = ['topics', 'subcategory', 'category', 'weekday', 'hour_group']
    name_idx_dict = {key: [i for i, col in enumerate(all_features) if col.startswith(key)] for key in singular_cols}
    numerical_cols = ['scroll_percentage', 'read_time', 'premium']
    name_idx_dict['numerical'] = [i for i, col in enumerate(all_features) if col in numerical_cols]
    impression_time_idx = all_features.index('impression_time_fixed')
        
    res = {key: ([], []) for key in name_idx_dict.keys()}

    for user_df in tqdm(df.partition_by('user_id', maintain_order=False)):
        x = user_df.drop('user_id').to_numpy()[0]
        x = np.array([np.array(x_i) for x_i in x])
        return x
                
        i = 0
        if i + w >= x.shape[1]:
            # in case history is shorter than the window then we pad it and select the last element as target
            pad_width = w - x[:, :-1].shape[1]
            pad_m = np.zeros((x.shape[0], pad_width))
            padded_x = np.concatenate((pad_m, x[:, :-1]), axis=1)
            y_i = x[:, -1]
            
            for key, idx in name_idx_dict.items():
                res[key][0].append(padded_x[idx, :].T)
                res[key][1].append(y_i[idx].T)
            
        else:
            while i + w < x.shape[1]:
                # in case history is larger than the window then we select the window and the target randomly between the next elements
                x_i = x[:, i:i+w]
                last_window_date = x_i[impression_time_idx][-1]
                max_telescope = max([t for t in range(i+w+1, x.shape[1]) if x[:, t][impression_time_idx].month == last_window_date.month and 
                                        x[:, t][impression_time_idx].day == last_window_date.day], default=i+w)
                print(i+w, max_telescope)
                target_random_id = np.random.randint(i+w, max_telescope+1)
                y_i = x[:, target_random_id]
                
                for key, idx in name_idx_dict.items():
                    res[key][0].append(x_i[idx, :].T)
                    res[key][1].append(y_i[idx].T)
                
                i+=stride
                         
            #TODO: add padding for the last sequence, if we want to keep it
                

    for key in res.keys():
        res[key] = (np.array(res[key][0]), np.array(res[key][1]))
    
    return res

In [36]:
x = np.array([1, 2.5, 3])
x = x.astype(np.float32)
x

array([1. , 2.5, 3. ], dtype=float32)

In [17]:
x = build_sequences(df[:100], w=10, stride=5)
x

  0%|          | 0/100 [00:00<?, ?it/s]


array([[106.0, 76.0, 115.0, ..., 71.0, 140.0, 43.0],
       [4, 10, 5, ..., 6, 5, 5],
       [2, 1, 1, ..., 2, 3, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [22]:
x[0].astype(np.float32)

array([106.,  76., 115., 127.,   2., 123.,  27.,  12.,   0., 202.,  77.,
        49.,  33.,  24.,  13.,  66.,  39.,  15., 277., 156.,  33., 212.,
       322., 109.,   0.,  54.,   0.,  39.,  29.,  56.,   1.,  31.,  29.,
       169.,  85.,  48., 130.,   6.,  92.,  22., 107.,  13., 227.,  37.,
        16.,  51., 376.,  26., 185.,   9., 116.,  80., 116.,  28.,  69.,
        16., 116.,  93.,  49., 100.,   0., 104.,  66.,  60.,  48., 196.,
        17., 183.,  61., 205.,  44.,   2., 116.,  45., 131.,  23., 143.,
        69.,  25.,  37.,  76.,  32., 120., 129.,  87.,  74.,   4.,  71.,
        68.,  64.,  17.,  85., 134., 249.,  31.,  43.,  10.,  34.,  56.,
        59.,  33., 194.,  70.,   0.,  32.,   0., 104.,  31., 145.,   3.,
        85.,  78., 165.,  31.,  38., 160.,  78., 576.,  26.,  61.,  51.,
        18.,  84.,  28.,  72.,  12.,   0.,  57.,  71., 140.,  43.],
      dtype=float32)

In [23]:
x[0]

array([106.0, 76.0, 115.0, 127.0, 2.0, 123.0, 27.0, 12.0, 0.0, 202.0,
       77.0, 49.0, 33.0, 24.0, 13.0, 66.0, 39.0, 15.0, 277.0, 156.0, 33.0,
       212.0, 322.0, 109.0, 0.0, 54.0, 0.0, 39.0, 29.0, 56.0, 1.0, 31.0,
       29.0, 169.0, 85.0, 48.0, 130.0, 6.0, 92.0, 22.0, 107.0, 13.0,
       227.0, 37.0, 16.0, 51.0, 376.0, 26.0, 185.0, 9.0, 116.0, 80.0,
       116.0, 28.0, 69.0, 16.0, 116.0, 93.0, 49.0, 100.0, 0.0, 104.0,
       66.0, 60.0, 48.0, 196.0, 17.0, 183.0, 61.0, 205.0, 44.0, 2.0,
       116.0, 45.0, 131.0, 23.0, 143.0, 69.0, 25.0, 37.0, 76.0, 32.0,
       120.0, 129.0, 87.0, 74.0, 4.0, 71.0, 68.0, 64.0, 17.0, 85.0, 134.0,
       249.0, 31.0, 43.0, 10.0, 34.0, 56.0, 59.0, 33.0, 194.0, 70.0, 0.0,
       32.0, 0.0, 104.0, 31.0, 145.0, 3.0, 85.0, 78.0, 165.0, 31.0, 38.0,
       160.0, 78.0, 576.0, 26.0, 61.0, 51.0, 18.0, 84.0, 28.0, 72.0, 12.0,
       0.0, 57.0, 71.0, 140.0, 43.0], dtype=object)

In [99]:
from polimi.utils._polars import reduce_polars_df_memory_size

window = 20
mask = 0
df_trucated = df.with_columns(
    pl.all().exclude('user_id').list.reverse().list.eval(pl.element().extend_constant(mask, window)).list.reverse().list.tail(window).name.keep()
)

df_trucated.head(2)

user_id,sentiment_label,category,premium,read_time,scroll_percentage,hour_group,weekday,topics_0,topics_1,topics_2,topics_3,topics_4,topics_5,topics_6,topics_7,topics_8,topics_9,topics_10,topics_11,topics_12,topics_13,topics_14,topics_15,topics_16,topics_17,topics_18,topics_19,topics_20,topics_21,topics_22,topics_23,topics_24,topics_25,topics_26,topics_27,topics_28,…,subcategory_137,subcategory_138,subcategory_139,subcategory_140,subcategory_141,subcategory_142,subcategory_143,subcategory_144,subcategory_145,subcategory_146,subcategory_147,subcategory_148,subcategory_149,subcategory_150,subcategory_151,subcategory_152,subcategory_153,subcategory_154,subcategory_155,subcategory_156,subcategory_157,subcategory_158,subcategory_159,subcategory_160,subcategory_161,subcategory_162,subcategory_163,subcategory_164,subcategory_165,subcategory_166,subcategory_167,subcategory_168,subcategory_169,subcategory_170,subcategory_171,subcategory_172,subcategory_173
u32,list[i8],list[i8],list[i8],list[f32],list[f32],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],…,list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8],list[i8]
2053634,"[1, 1, … 1]","[25, 4, … 4]","[0, 0, … 0]","[0.0, 116.0, … 0.0]","[0.0, 100.0, … 0.0]","[4, 0, … 5]","[1, 2, … 3]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"
1899092,"[3, 1, … 3]","[5, 25, … 4]","[0, 0, … 0]","[60.0, 17.0, … 8.0]","[100.0, 24.0, … 60.0]","[4, 4, … 1]","[3, 3, … 4]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[0, 0, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]",…,"[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]"


In [100]:
behaviors_df = pl.read_parquet('/home/ubuntu/dset_complete/subsample/train_ds.parquet')
behaviors_df.head(2)

impression_id,user_id,article,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,constrastive_emb_icm_l_inf_article,std_article_kenneth_emb_icm,std_article_distilbert_emb_icm,std_article_bert_emb_icm,std_article_roberta_emb_icm,std_article_w_to_vec_emb_icm,std_article_emotions_emb_icm,std_article_constrastive_emb_icm,skew_article_kenneth_emb_icm,skew_article_distilbert_emb_icm,skew_article_bert_emb_icm,skew_article_roberta_emb_icm,skew_article_w_to_vec_emb_icm,skew_article_emotions_emb_icm,skew_article_constrastive_emb_icm,kurtosis_article_kenneth_emb_icm,kurtosis_article_distilbert_emb_icm,kurtosis_article_bert_emb_icm,kurtosis_article_roberta_emb_icm,kurtosis_article_w_to_vec_emb_icm,kurtosis_article_emotions_emb_icm,kurtosis_article_constrastive_emb_icm,entropy_article_kenneth_emb_icm,entropy_article_distilbert_emb_icm,entropy_article_bert_emb_icm,entropy_article_roberta_emb_icm,entropy_article_w_to_vec_emb_icm,entropy_article_emotions_emb_icm,entropy_article_constrastive_emb_icm,kenneth_emb_icm_minus_median_article,distilbert_emb_icm_minus_median_article,bert_emb_icm_minus_median_article,roberta_emb_icm_minus_median_article,w_to_vec_emb_icm_minus_median_article,emotions_emb_icm_minus_median_article,constrastive_emb_icm_minus_median_article,impression_time
u32,u32,i32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,datetime[μs]
149474,139836,9778728,0,2,13.0,null,false,2,null,false,5,150,521,836,419,3,7,0.287908,0.179426,0.880068,false,142,0.9654,"""Negative""",1,5,18,251,7,22415,220247,1.004828e6,0.101772,"""article_default""",0,0,…,0.017241,0.003313,0.010775,0.020001,6.258241,0.005474,4.929348,0.062682,1.300164,2.060066,1.152196,1.754251,2.088241,1.214502,1.406736,1.339344,5.427761,1.025453,3.529418,5.37705,1.054931,1.963193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002187,-0.004914,-0.014626,-3.694031,-0.002645,-3.802204,-0.039383,2023-05-24 07:47:53
149474,139836,9778669,0,2,13.0,null,false,2,null,false,5,85,199,313,266,3,7,0.427136,0.271565,0.336149,false,118,0.9481,"""Negative""",1,5,11,150,4,74491,373488,4.365609e6,0.199447,"""article_default""",0,1,…,0.017094,0.003318,0.011131,0.019302,5.737217,0.003193,8.05171,0.06661,1.08501,1.063945,0.947301,1.095085,1.058148,1.020884,1.072975,0.822594,0.947987,0.355356,0.968488,1.051799,0.600099,0.791561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002933,-0.010537,-0.017286,-4.617104,-0.002901,-7.678095,-0.057551,2023-05-24 07:47:53


In [101]:
def build_sequences_cls_iterator(history_seq: pl.DataFrame, behaviors: pl.DataFrame, window:int):
    mask = 0
    history_seq_trucated = history_seq.with_columns(
        pl.all().exclude('user_id').list.reverse().list.eval(pl.element().extend_constant(mask, window)).list.reverse().list.tail(window).name.keep()
    )
    
    for user_id, user_history in tqdm(history_seq_trucated.partition_by(['user_id'], as_dict=True, maintain_order=False).items()): #order not maintained
        for b in behaviors.filter(pl.col('user_id') == user_id[0]).iter_slices(1):
            yield (b.drop('target'), user_history, b['target'].item())
        

In [103]:
for i, (features, user_history_trunc, target) in enumerate(build_sequences_cls_iterator(df, behaviors_df, window=20)):
    print(target)
    break

  0%|          | 0/15143 [00:00<?, ?it/s]


0
